SNEWS2.0 White Paper Event Totals Table
=================================

This notebook takes a model from the SNEWPY library and generates a time-independent snowglobes fluence file from the data. It then runs it through snowglobes (must be installed) and returns the summative counts, a routine to distill output to total counts is also included.

In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from snewpy.models import *
import os
from shutil import copyfile
from astropy.io import ascii

In [2]:
def OutputInSnowGlobesFormat(SN,savefile="SNOformat.dat",d=10.):
    '''
    Write to disk a SnowGlobes fluence file summed over the entire SNEWPY model \
    Parameters
    ----------
    SN : SupernovaModel
        Supernova model from the SNEWPY library.  
    savefile : string
        path to file for the snowglobes fluence file
    d : float
        distance (in kpc) to place the model for the fluence calculation
    '''
    d *= 1000.*3.086e+18 #convert from kpc to cm
    keV=1e3
    MeV=1e6
    GeV=1e9
    
    times=SN.get_time()
    dt = np.zeros(len(times))
    for i in range(len(times)-1):
        dt[i] = times[i+1]-times[i]
    dt[len(times)-1] = times[-1]-times[-2]
    
    nenergybins = 501
    E=np.linspace(0,100,nenergybins)*MeV
    
    finalOF=np.zeros(len(Flavor)*len(E)).reshape(len(Flavor),len(E))
    OscillatedFluence = {}
    OscillatedSpectra = SN.get_oscillatedspectra(times[0],E)
    for flavor in range(len(Flavor)):
        OscillatedFluence[flavor] = OscillatedSpectra[flavor]*dt[0]* 200.*keV  / (4.*np.pi*d**2)
    for i in range(1,len(times)):
        OscillatedSpectra = SN.get_oscillatedspectra(times[i],E)
        for flavor in range(len(Flavor)):
            OscillatedFluence[flavor] += OscillatedSpectra[flavor]*dt[i]* 200.*keV  / (4.*np.pi*d**2)
            
    file = open(savefile,"w")
    
    for i in range(0,nenergybins):
        file.write("{0:.4f}".format(E[i]/GeV))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_e][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_e_bar][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x_bar][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x_bar][i])+"\n")

    file.close()

In [3]:
def get_SGresults(SGdata,SG_settings,targettype,mass,eventtype="total"):
    '''
    small parser for the output from the snowglobes routine: make_event_table.pl
    
    Parameters
    ==========
    SGdata : dict
        dictionary of output (after splitlines) from the make_event_table.pl routine from snowglobes
    SG_settings : dict
        dictionary of settings describing the SGdata
    targettype : string
        dictionary identifier of the results wanted
    mass : float
        mass to scale snowglobes output to for the results wanted
    eventtype : string
        discriminates between total events, elastic scattering events or neutral current events,
        there is currently no individual channel parsing, although this data is in SGdata
    '''
    if eventtype == 'es': 
        line = -3
    elif eventtype == 'nc': 
        line = -2
    elif eventtype == 'total': 
        line = -1
    else:
        print("Warning, eventtype unknown, returning total")
        line = -1
    return float(SGdata[targettype][line].split(":")[-1])*mass/SG_settings[targettype][2]
        

Generate Fluence Files
==================

Here we generate snowglobes fluence files for three models and under three oscillation scenarios


In [4]:
snmodel = Bollig2016("../../models/Bollig_2016/s11.2c", NoTransformation(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s11.2c", AdiabaticMSW_IMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s11.2c", AdiabaticMSW_NMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat.dat",d=10.)

snmodel = Bollig2016("../../models/Bollig_2016/s27.0c", NoTransformation(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s27.0c", AdiabaticMSW_IMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s27.0c", AdiabaticMSW_NMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat.dat",d=10.)

snmodel = OConnor2015("../../models/OConnor_2015/M1_neutrinos.dat", NoTransformation(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat.dat",d=10.)
snmodel = OConnor2015("../../models/OConnor_2015/M1_neutrinos.dat", AdiabaticMSW_IMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat.dat",d=10.)
snmodel = OConnor2015("../../models/OConnor_2015/M1_neutrinos.dat", AdiabaticMSW_NMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat.dat",d=10.)

Execute SNOwGLoBES
==================
Here we run SNOwGLoBES to determine total event rates in default detectors.  We store the summed data in order to use it for arbitrary detectors in the following

In [11]:
SG_DIR="/Users/evanoc/Documents/GitHub/snowglobes/" #pr/10 on July 20, 2020
SNEWPY_DIR="/Users/evanoc/research/supernova_models/"
filebase = {"OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat",
            "OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat",
            "OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat",
            "Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat",
            "Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat",
            "Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat",
            "Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat",
            "Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat",
            "Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat"}

SG_settings={}
SG_settings["WC"]        = ["water", "wc100kt30prct",100]
SG_settings["argon"]     = ["argon", "ar40kt",40]
SG_settings["lead1"]      = ["lead", "halo1",0.079]
SG_settings["lead2"]      = ["lead", "halo2",1]
SG_settings["scint"]     = ["scint", "scint20kt",20]
SG_settings["nova_soup"] = ["nova_soup", "novaFD",14]
SG_settings['icecube']   = ["water", "icecube", 51600]
SG_settings['km3net']   = ["water_nooxygen", "km3net", 69366]

allSGdata = {}
allSGdataUW = {}
for SGfile in filebase:
    print("Working on",SGfile,"...")
    SGdata = {}
    SGdataUW = {}
    for type in SG_settings:
        material = SG_settings[type][0]
        detector = SG_settings[type][1]
        os.chdir(SNEWPY_DIR+"doc/nb")
        copyfile(SGfile+'.dat', SG_DIR+"fluxes/"+SGfile+'.dat')
        os.chdir(SG_DIR)
        command = "./supernova.pl "+SGfile+" "+material+" "+detector
        os.system(command)
        command = "./make_event_table.pl  "+SGfile+" "+material+" "+detector
        SGdata[type] = os.popen(command).read().splitlines()
        command = "./make_event_table.pl  "+SGfile+" "+material+" "+detector+" 1"
        SGdataUW[type] = os.popen(command).read().splitlines()
        os.chdir(SNEWPY_DIR+"doc/nb")
    allSGdata[SGfile] = SGdata
    allSGdataUW[SGfile] = SGdataUW


Working on OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat ...
Working on OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat ...
Working on Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat ...
Working on OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat ...
Working on Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat ...
Working on Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat ...
Working on Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat ...
Working on Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat ...
Working on Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat ...


Generate table for SNEWS2.0 white paper
=================================

This will need style changes, but allows for easy recomputing if models change

In [20]:
def round_to_2(x):
    if x==0: 
        return 0
    else:
        return round(x, -int(np.floor(np.log10(np.abs(x))))+1)

filebase = {0:"OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat",
            1:"OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat",
            2:"OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat",
            3:"Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat",
            4:"Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat",
            5:"Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat",
            6:"Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat",
            7:"Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat",
            8:"Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat"}

det_maps = {"Super-K":"WC","Hyper-K":"WC","IceCube":"icecube","KM3NeT":"km3net",
            "KamLAND":"scint","Borexino":"scint","JUNO":"scint","SNO+":"scint","NO{$\\nu$}A":"nova_soup",
            "HALO":"lead1","HALO-1kT":"lead2","DUNE":"argon","MicroBooNe":"argon","SBND":"argon"}


data = {}
data['Experiment'] = ['Super-K','Hyper-K','IceCube','KM3NeT','KamLAND','Borexino',
                       'JUNO','SNO+','NO{$\\nu$}A','HALO','HALO-1kT','DUNE','MicroBooNe','SBND']
data['Type'] = ["H$_2$O/$\\bar{\\nu}_e$","H$_2$O/$\\bar{\\nu}_e$","String/$\\bar{\\nu}_e$",
                "String/$\\bar{\\nu}_e$",'C$_n$H$_{2n}$/$\\bar{\\nu}_e$',
                'C$_n$H$_{2n}$/$\\bar{\\nu}_e$','C$_n$H$_{2n}$/$\\bar{\\nu}_e$',
                'C$_n$H$_{2n}$/$\\bar{\\nu}_e$','C$_n$H$_{2n}$/$\\bar{\\nu}_e$',
                "Lead/$\\nu_e$","Lead/$\\nu_e$","Ar/$\\nu_e$","Ar/$\\nu_e$","Ar/$\\nu_e$"]
data['Mass [kt]'] = [32,220,51600,69366*3,1,0.278,20,0.7,14,0.079,1,40,0.09,0.12]
data['Location'] = ["Japan","Japan","South Pole","Italy/France","Japan","Italy","China",
                    "Canada","USA","Canada","Italy","USA","USA","USA"]
data['\\SI{11.2}{\solarmass}'] = []
data['\\SI{27.0}{\solarmass}'] = []
data['\\SI{40.0}{\solarmass}'] = []
for experiment in range(len(data['Experiment'])):
    mass = data['Mass [kt]'][experiment]
    dettype = det_maps[data['Experiment'][experiment]]
    
    counts_LCN = int(get_SGresults(allSGdata[filebase[4]],SG_settings,dettype,mass))
    counts_LCI = int(get_SGresults(allSGdata[filebase[5]],SG_settings,dettype,mass))
    counts_MCN = int(get_SGresults(allSGdata[filebase[7]],SG_settings,dettype,mass))
    counts_MCI = int(get_SGresults(allSGdata[filebase[8]],SG_settings,dettype,mass))
    counts_HCN = int(get_SGresults(allSGdata[filebase[1]],SG_settings,dettype,mass))
    counts_HCI = int(get_SGresults(allSGdata[filebase[2]],SG_settings,dettype,mass))

    post = ['','','','','','']
    if counts_LCN>10000: 
        counts_LCN = int(counts_LCN/1000.0+0.5)
        post[0] = 'K'
    if counts_MCN>10000: 
        counts_MCN = int(counts_MCN/1000+0.5)
        post[2] = 'K'
    if counts_HCN>10000: 
        counts_HCN = int(counts_HCN/1000+0.5)
        post[4] = 'K'
    if counts_LCI>10000: 
        counts_LCI = int(counts_LCI/1000+0.5)
        post[1] = 'K'
    if counts_MCI>10000: 
        counts_MCI = int(counts_MCI/1000+0.5)
        post[3] = 'K'
    if counts_HCI>10000: 
        counts_HCI = int(counts_HCI/1000+0.5)
        post[5] = 'K'
        
    data['\\SI{11.2}{\solarmass}'].append(str(round_to_2(counts_LCN))+post[0]+"/"+str(round_to_2(counts_LCI))+post[1])
    data['\\SI{27.0}{\solarmass}'].append(str(round_to_2(counts_MCN))+post[2]+"/"+str(round_to_2(counts_MCI))+post[3])
    data['\\SI{40.0}{\solarmass}'].append(str(round_to_2(counts_HCN))+post[4]+"/"+str(round_to_2(counts_HCI))+post[5])


    
#Hacking in different numbers

#IceCube, the effective mass in SNOwGLoBES is artificially high.  This is because the
#non-standard energy dependence is handled through the efficiencies.  To get an effective
#mass we take the ratio of the total weighted events to the unweigthed events and multiply
#the unweigthed mass (the entry in SNOwGLoBES), see below for details.  Here we take the
#effective mass of the s27 normal scenerio and discuss the range in the table caption

dettype='icecube'
mass=51600
data['Mass [kt]'][2] = "~"+str(100*int(0.01*(mass*get_SGresults(allSGdata[filebase[7]],SG_settings,dettype,mass)/
      get_SGresults(allSGdataUW[filebase[7]],SG_settings,dettype,mass)+50)))+"*"

dettype='km3net'
mass=69366*3
data['Mass [kt]'][3] = "~"+str(10*int(0.1*(mass*get_SGresults(allSGdata[filebase[7]],SG_settings,dettype,mass)/
      get_SGresults(allSGdataUW[filebase[7]],SG_settings,dettype,mass)+5)))+"*"


#DarkSide-20k & Ar/any $\nu$ & 0.02 & Italy & \\
#For DarkSide-20k, the text on the SNEWS2.0 paper say the s27 will produce 250 events at 10kpc with a
#detector mass of 38.6 tonnes
s27_epT = 6476.6839378239

mass = 0.0386
data['Experiment'].append('DarkSide-20k')
data['Type'].append('Ar/any $\\nu$')
data['Mass [kt]'].append(mass)
data['Location'].append('Italy')
data['\\SI{11.2}{\solarmass}'].append("-")
data['\\SI{27.0}{\solarmass}'].append(int(mass*s27_epT))
data['\\SI{40.0}{\solarmass}'].append("-")


#For XENONnT, LZ, and PandaX-4T, we take the values from Lang et al. [Phys. Rev. D 94 (2016) no.10, 103009].
#We take the S2-only, 60PE threshold numbers for the 11.2 msun and 27 msun modeels as they
#are the ones used here. We take 9.4 events/tonne for the 11.2 model and 17.6 events/tonne for the 27.0 model
s11_epT = 9400.
s27_epT = 17600.

mass = 0.008
data['Experiment'].append('XENONnT')
data['Type'].append('Xe/any $\\nu$')
data['Mass [kt]'].append(mass)
data['Location'].append('Italy')
data['\\SI{11.2}{\solarmass}'].append(int(mass*s11_epT))
data['\\SI{27.0}{\solarmass}'].append(int(mass*s27_epT))
data['\\SI{40.0}{\solarmass}'].append("-")

mass = 0.007
data['Experiment'].append('LZ')
data['Type'].append('Xe/any $\\nu$')
data['Mass [kt]'].append(mass)
data['Location'].append('USA')
data['\\SI{11.2}{\solarmass}'].append(int(mass*s11_epT))
data['\\SI{27.0}{\solarmass}'].append(int(mass*s27_epT))
data['\\SI{40.0}{\solarmass}'].append("-")

mass = 0.004
data['Experiment'].append('PandaX-4T')
data['Type'].append('Xe/any $\\nu$')
data['Mass [kt]'].append(mass)
data['Location'].append('China')
data['\\SI{11.2}{\solarmass}'].append(int(mass*s11_epT))
data['\\SI{27.0}{\solarmass}'].append(int(mass*s27_epT))
data['\\SI{40.0}{\solarmass}'].append("-")


ascii.write(data,Writer=ascii.Latex)

\begin{table}
\begin{tabular}{ccccccc}
Experiment & Type & Mass [kt] & Location & \SI{11.2}{\solarmass} & \SI{27.0}{\solarmass} & \SI{40.0}{\solarmass} \\
Super-K & H$_2$O/$\bar{\nu}_e$ & 32 & Japan & 4000/4100 & 7800/7600 & 7600/4900 \\
Hyper-K & H$_2$O/$\bar{\nu}_e$ & 220 & Japan & 28K/28K & 53K/52K & 52K/34K \\
IceCube & String/$\bar{\nu}_e$ & ~2500* & South Pole & 320K/330K & 660K/660K & 820K/630K \\
KM3NeT & String/$\bar{\nu}_e$ & ~150* & Italy/France & 17K/18K & 37K/38K & 47K/38K \\
KamLAND & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 1 & Japan & 190/190 & 360/350 & 340/240 \\
Borexino & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 0.278 & Italy & 52/52 & 100/97 & 96/65 \\
JUNO & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 20 & China & 3800/3800 & 7200/7000 & 6900/4700 \\
SNO+ & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 0.7 & Canada & 130/130 & 250/240 & 240/160 \\
NO{$\nu$}A & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 14 & USA & 1900/2000 & 3700/3600 & 3600/2500 \\
HALO & Lead/$\nu_e$ & 0.079 & Canada & 4/3 & 9/8 & 9/9 \\
HALO-1kT & Lead/$\nu_